# Лабораторная работа 3.
# Сетевые алгоритмы. Динамические алгоритмы поиска путей.


## Выполнил студент группы БФИ2302 Филимонов Михаил Алексеевич
***

### Задание

1.  Реализовать алгоритм поиска кратчайшего расстояния между двумя вершинами ориентированного взвешенного графа в соответствии с вариантом.

2.  Предусмотреть задание графа в виде матрицы смежности/инцидентности, читаемой из файла, либо графически с помощью пользовательского интерфейса.

3.  Разработать графический интерфейс пользователя с визуализацией графа и отображением кратчайшего расстояния между задаваемыми пользователем вершинами.

4. По результатам работы проанализировать временную сложность работы заданного алгоритма в зависимости от числа узлов и ребер графа.
Данные представить в виде таблицы.



### Алгоритмы:

Алгоритм Флойда-Уоршелла| Алгоритм Дейкстры | Алгоритм Беллмана-Форда | Алгоритм Джонсона| Алгоритм Левита | Алгоритм Йена



In [1]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import time
import heapq
from IPython.display import display


In [2]:
#Алгоритм Флойда-Уоршелла (поиск кратчайших путей в графах)
def floyd_warshall(adj_matrix): # Ф-ия принимает на вход матрицу смежности 
    num_nodes = len(adj_matrix) # Определяет количество вершин в графе как размерность матрицы смежности.
    dist = adj_matrix.copy() # Создает копию матрицы смежности и сохраняет её,используется для хранения оценок кратчайших расстояний между всеми парами вершин
    for k in range(num_nodes): # цикл перебирает все вершины графа, от 0 до num_nodes - 1 (k пром)
        for i in range(num_nodes): # цикл перебирает все вершины графа, от 0 до num_nodes - 1 (i нач)
            for j in range(num_nodes): # цикл перебирает все вершины графа, от 0 до num_nodes - 1 (j кон)
                if dist[i][k] and dist[k][j]: # Проверяет, существует ли путь от вершины i до вершины k и от вершины k до вершины j 
                    new_distance = dist[i][k] + dist[k][j] # Если путь от i до k и от k до j существует, вычисляется расстояние от i до j через вершину k
                    if dist[i][j] == 0 or new_distance < dist[i][j]: # Проверяет, является ли путь от i до j через k короче, чем известный кратчайший путь от i до j. dist[i][j] == 0 означает, что ранее путь между вершинами i и j считался бесконечным
                        dist[i][j] = new_distance # Если путь через k короче, обновляет dist[i][j] новым, более коротким расстоянием
    return dist

In [16]:
#Алгоритм Дейкстры (поиск кратчайших путей от одной вершины графа ко всем остальным)
def dijkstra(adj_matrix, start): # Ф-ия принимает на вход матрицу смежности
    num_nodes = len(adj_matrix) #  Определяет количество вершин в графе
    distances = {node: float('inf') for node in range(num_nodes)} # Создает словарь, который будет хранить кратчайшие расстояния от начальной вершины до каждой вершины
    distances[start] = 0 # Устанавливает расстояние от начальной вершины до самой себя равным 0.
    priority_queue = [(0, start)] # Создает приоритетную очередь priority_queue (реализованную с помощью кучи) для хранения вершин, которые нужно посетить
    while priority_queue: # Основной цикл алгоритма. Он выполняется до тех пор, пока приоритетная очередь не станет пустой
        current_distance, current_node = heapq.heappop(priority_queue) # Извлекает из приоритетной очереди вершину с наименьшим текущим расстоянием
        if current_distance > distances[current_node]: # Проверяет, не является ли текущее расстояние до вершины больше, чем уже известное кратчайшее расстояние
            continue
        for neighbor in range(num_nodes): # Перебирает всех соседей текущей вершины
            weight = adj_matrix[current_node][neighbor] # Получает вес ребра между данными знач из матрицы смежности.
            if weight > 0: # Проверяет, существует ли ребро между данными знач. Вес больше 0 предполагает существование ребра.
                distance = current_distance + weight # Вычисляет расстояние от начальной вершины до neighbor через current_node.
                if distance < distances[neighbor]: # Проверяет, является ли вычисленное расстояние меньше, чем известное расстояние до данного знач
                    distances[neighbor] = distance # Если новый путь короче, обновляет расстояние до данного знач в словаре
                    heapq.heappush(priority_queue, (distance, neighbor)) # Добавляет данное знач в приоритетную очередь с новым расстоянием.
    return distances

In [17]:
#Алгоритм Беллмана-Форда (поиск кратчайших путей от одной вершины графа до всех остальных)
def bellman_ford_fixed(adj_matrix, start): # Ф-ия принимает на вход матрицу смежности и индекс начальной вершины
    num_nodes = len(adj_matrix) # Определяет количество вершин в графе
    distances = {node: float('inf') for node in range(num_nodes)} # Создает словарь, который будет хранить кратчайшие расстояния от начальной вершины до каждой вершины
    distances[start] = 0 # Устанавливает расстояние от начальной вершины до самой себя равным 0.
    for _ in range(num_nodes - 1): # цикл, который повторяется num_nodes - 1 раз
        for u in range(num_nodes): # цикл перебирает все вершины графа, рассматривая их как источник ребер
            for v in range(num_nodes): # цикл перебирает все вершины графа, рассматривая их как целевые вершины ребер
                weight = adj_matrix[u][v] # Получает вес ребра между данными вершинами
                if weight > 0 and distances[u] != float('inf') and distances[u] + weight < distances[v]: # Если существует ребро из u в v и если достижима вершина u из начальной вершины и если можно улучшить расстояние до вершины v через ребро (u, v)
                    distances[v] = distances[u] + weight # То, обновляет расстояние до вершины v
    return {k: (v if v != float('inf') else 0) for k, v in distances.items()} # Строка исправляет ситуацию, когда вершина недостижима, и заменяет float('inf') на 0

In [18]:
#Алгоритм Джонсона (поиск кратчайших путей между всеми парами вершин взвешенного ориентированного графа)
def johnson_fixed(adj_matrix): # Ф-ия принимает на вход матрицу смежности 
    num_nodes = len(adj_matrix) # Определяет количество вершин в графе
    new_matrix = np.zeros((num_nodes + 1, num_nodes + 1)) # Создает новую матрицу смежности размером (num_nodes + 1) x (num_nodes + 1), заполненную нулями. Эта матрица будет использоваться для добавления новой вершины s в граф
    new_matrix[:num_nodes, :num_nodes] = adj_matrix # Копирует исходную матрицу смежности в верхний левый угол новой матрицы
    for i in range(num_nodes): #  Добавляет новую вершину s (с индексом num_nodes) в граф
        new_matrix[num_nodes, i] = 0 # Ребра от s ко всем остальным вершинам имеют вес 0
    h = bellman_ford_fixed(new_matrix, num_nodes) # Вызывает ф-ию для вычисления расстояний от новой вершины s до всех остальных вершин в расширенном графе
    new_adj_matrix = adj_matrix.copy() # Создает копию исходной матрицы смежности
    for u in range(num_nodes): # цикл перебирает все вершины графа, рассматривая их как источник ребер
        for v in range(num_nodes): # цикл перебирает все вершины графа, рассматривая их как целевые вершины ребер
            if new_adj_matrix[u, v] > 0 and u in h and v in h: # Если существует ребро от вершины u к вершине v и мы убеждаемся, что у вершин u и v есть вычисленный потенциал h
                new_adj_matrix[u, v] += h[u] - h[v] # То, Пересчитывает вес ребра от вершины u к вершине v с использованием потенциалов h
    distances = np.zeros((num_nodes, num_nodes)) #  Создает матрицу размером num_nodes x num_nodes для хранения кратчайших расстояний между всеми парами вершин
    for u in range(num_nodes): # цикл, который перебирает все вершины u графа. Для каждой вершины u вычисляются кратчайшие пути до всех остальных вершин
        distances[u] = list(dijkstra(new_adj_matrix, u).values()) # список расстояний присваивается элементу distances[u], тобеж словарю
    return distances

In [19]:
#Алгоритм Левита (поиск кратчайшего расстояние от одной из вершин графа до всех остальных)
def levit(adj_matrix, start): # Определяет ф-ию, принимающую матрицу смежности и начальную вершину в качестве аргументов
    num_nodes = len(adj_matrix) # Определяет количество вершин в графе
    distances = {i: float('inf') for i in range(num_nodes)} # Создает словарь, где ключами являются номера вершин (от 0 до num_nodes - 1)
    distances[start] = 0 # Расстояние от начальной вершины до самой себя устанавливается в 0
    main_queue = [] # Создает пустой список, тобеж очередь для хранения вершин, которые нужно посетить
    urgent_queue = [] # Создает пустой список, тобеж очередь для хранения вершин, которые нужно обработать приоритетно
    main_queue.append(start) # Добавляет начальную вершину в main_queue,чтобы пошёл поиск кратчайших путей
    while main_queue or urgent_queue: # цикл работает до тех пор, пока обе очереди не станут пустыми
        if urgent_queue: # есть ли вершины в urgent_queue
            u = urgent_queue.pop(0) #  Если urgent_queue не пуста, извлекает первую вершину (u) из urgent_queue
        else:
            u = main_queue.pop(0) # Извлекает первую вершину (u) из main_queue
        for v in range(num_nodes): # Перебирает все вершины v в графе
            weight = adj_matrix[u][v] # Получает вес ребра между вершинами u и v из матрицы смежности
            if weight > 0: # Если вес больше 0
                if distances[v] == float('inf'): # Если расстояние до вершины v все еще равно бесконечности
                    distances[v] = distances[u] + weight # Обновляет расстояние до вершины v как сумму расстояния до вершины u и веса ребра между u и v
                    main_queue.append(v) # Добавляет вершину v в main_queue, чтобы была обработана позже
                elif distances[v] > distances[u] + weight: # Если текущее расстояние до v больше, чем расстояние до u плюс вес ребра между u и v
                    distances[v] = distances[u] + weight # Обновляет расстояние до вершины v более коротким значением
                    urgent_queue.append(v) # Добавляет вершину v в urgent_queue, ее соседей нужно пересмотреть приоритетно
    return distances 

In [20]:
#Алгоритм Йена (поиск K-кратчайших путей без циклов в графе с неотрицательными длинами рёбер)
import heapq

def yen_k_shortest_paths(graph, start, end, k=3): # Определяет ф-ию, которая должна находить k кратчайших путей между вершинами start и end в графе graph
    def dijkstra_modified(g, src, dst, forbidden_edges): # Определяет ф-ию, модифицированная версия алгоритма Дейкстры. Она находит кратчайший путь между вершинами в графе g, избегая указанных ребер в forbidden_edges 
        n = len(g) # Определяет количество вершин в графе g
        dist = [float('inf')] * n # Создает список для хранения расстояний от начальной вершины (src) до всех остальных вершин
        dist[src] = 0 # Расстояние от начальной вершины до самой себя устанавливается в 0
        prev = [-1] * n # Создает список для хранения предыдущей вершины на кратчайшем пути от src до каждой вершины [предыдущая вершина пока не известна]
        heap = [(0, src)] # Создает приоритетную очередь (кучу), куча содержит пары (d, u), где d - это текущее расстояние от src до вершины u. Изначально в кучу добавляется только начальная вершина src с расстоянием 0
        
        while heap: # цикл работает до тех пор, пока куча не станет пустой
            d, u = heapq.heappop(heap) # Извлекает вершину u с наименьшим расстоянием d из кучи. Далее удаляя и возвращая наименьший элемент из кучи
            if u == dst: # Если извлеченная вершина u является целевой вершиной dst
                break
            if d > dist[u]: # Если текущее расстояние d до вершины u, извлеченной из кучи, больше, чем уже кратчайшее расстояние dist[u]
                continue
            for v in range(n): # Перебирает все соседние вершины v вершины u
                if g[u][v] > 0 and (u, v) not in forbidden_edges: # Если существет вес ребра больше 0 и если не запрещено ребро (u, v)
                    if dist[v] > dist[u] + g[u][v]: # Если найдено более короткое расстояние до вершины v через вершину u
                        dist[v] = dist[u] + g[u][v] # Обновляет расстояние до вершины v
                        prev[v] = u # Устанавливает вершину u как предыдущую вершину на кратчайшем пути до v
                        heapq.heappush(heap, (dist[v], v)) # Добавляет вершину v в кучу с новым расстоянием
        
        if dist[dst] == float('inf'): # После завершения цикла проверяет, было ли найдено какое-либо расстояние до целевой вершины dst
            return None
        path = [] # Если путь найден, создает пустой список для хранения вершин на кратчайшем пути
        u = dst # Начинает построение пути с целевой вершины dst
        while u != -1: # Выполняет цикл, пока не достигнет начальной вершины src 
            path.append(u) # Добавляет текущую вершину u в путь
            u = prev[u] # Переходит к предыдущей вершине на пути
        return path[::-1] # Возвращает путь в обратном порядке от начальной вершины до целевой вершины

    # (Первый путь)
    A = [dijkstra_modified(graph, start, end, set())] # Находит первый кратчайший путь между вершинами start и end в графе graph с использованием функции 
    if not A[0]: # Проверяет, был ли найден какой-либо путь
        return [] # ф-ия возвращает пустой список, если путь не найден
    
    #(Куча для кандидатов)
    B = [] # пустой список

    for _ in range(1, k): # цикл выполняется k-1 раз (начиная с 1), чтобы найти оставшиеся k-1 кратчайших путей
        for i in range(len(A[-1]) - 1): # цикл перебирает все ребра в последнем найденном кратчайшем пути
            spur_node = A[-1][i] # Определяет i-ую вершину в последнем найденном кратчайшем пути, от которой будет искаться альтернативный путь
            root_path = A[-1][:i+1] # Определяет часть последнего найденного кратчайшего пути от начальной вершины до spur_node
            
            # (Запрещаем рёбра из предыдущих путей)
            banned_edges = set() # Инициализирует множество для хранения запрещенных ребер
            for p in A: # цикл перебирает все уже найденные кратчайшие пути в списке
                if len(p) > i and p[:i+1] == root_path and i+1 < len(p): # Если длинный, чтобы иметь вершину с индексом i и если имеет корневой путь, что и текущий путь и если у пути есть следующее ребро после spur_node
                    banned_edges.add((p[i], p[i+1])) # Если всё выполнено, добавляет ребро, следующее за spur_node в текущем пути p, в множество
            
            spur_path = dijkstra_modified(graph, spur_node, end, banned_edges) # Находит от spur_node до конечной вершины end, избегая указанных ребер
            if spur_path:
                full_path = root_path[:-1] + spur_path # Если spur_path был найден, создает полный путь (full_path), объединяя root_path
                cost = sum(graph[full_path[j]][full_path[j+1]] for j in range(len(full_path)-1)) # Вычисляет стоимость full_path как сумму весов ребер вдоль пути
                heapq.heappush(B, (cost, full_path)) # Добавляет full_path в кучу B с ключом, равным стоимости пути
        
        if not B: # Проверка пуста ли куча
            break
        A.append(heapq.heappop(B)[1]) # Извлекает путь с наименьшей стоимостью из кучи B, удаляет и возвращает элемент с наименьшим ключом, путь добавляется в список A

    return A[:k] # Возвращает первые k путей из списка A

In [21]:
#Анализ производительности
def analyze_final_corrected_algorithms(adj_matrix): # Определяет ф-ию, принимающую матрицу смежности
    num_nodes = len(adj_matrix) # Определяет количество вершин в графе, используя размер матрицы смежности
    algorithms = {
        'Флойда-Уоршелла': floyd_warshall,
        'Дейкстры': lambda mat: dijkstra(mat, 0),
        'Беллмана-Форда': lambda mat: bellman_ford_fixed(mat, 0),
        'Джонсона': johnson_fixed,
        'Левита': lambda mat: levit(mat, 0),
        'Йена': lambda mat: yen_k_shortest_paths(mat, 0, len(mat), 3)
    } # Создает словарь, который связывает названия алгоритмов (строки) с соответствующими функциями
    results = [] # пустой список
    for algo_name, algo_func in algorithms.items(): # Перебирает данные пары в словаре algorithms
        start_time = time.perf_counter_ns() # Измеряет время начала выполнения алгоритма
        elapsed_time = time.perf_counter_ns() - start_time
        results.append((algo_name, num_nodes, elapsed_time)) # Добавляет пакет с информацией об алгоритме в список
    df = pd.DataFrame(results, columns=['Алгоритм', 'Ноды', 'Время (ns)']) # Создает pandas DataFrame из списка results
    return df

In [22]:
#Определение матрицы смежности тестов
with open("graphLab3.txt", "r") as file:
    source = file.read().strip().split("\n")
    test_adj_matrix = [] # пустой список
    for line in source: # Перебирает каждую строку в списке source
        a = [int(x) for x in line.split(" ")] # Разделяет строку line на отдельные числа и преобразует их в целые числа
        test_adj_matrix.append(a) # Добавляет список целых чисел a (представляющий одну строку матрицы) в список
showed_test_adj_matrix = test_adj_matrix # Создает копию списка, предназначеную для использования с библиотекой, использующаяся для визуализации графа

print("Test matrix")
print(*test_adj_matrix, sep="\n", end="\n") # Выводит содержимое матрицы смежности

test_adj_matrix = np.array(test_adj_matrix) # Преобразует список списков


Test matrix
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 2, 0, 0, 0, 0, 5, 0, 0, 0]
[0, 2, 0, 3, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 3, 0, 4, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 4, 0, 5, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 5, 0, 6, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 6, 0, 7, 0, 7, 0]
[0, 0, 0, 0, 0, 0, 7, 0, 8, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 8, 0, 9, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 9, 0, 10]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 0]


In [27]:
df_final_corrected_analysis = analyze_final_corrected_algorithms(test_adj_matrix) # Вызывает ф-ию, передавая ей в качестве аргумента данную переменную
display(df_final_corrected_analysis) # Отображает DataFrame

from pyvis import network as net # Импортирует модуль network из библиотеки визуальных графов, используя псевдоним net
import os

g = net.Network(notebook=True, bgcolor="#FFFFFF", cdn_resources="remote") # Создает объект Network из библиотеки pyvis
# Добавляет вершины в граф
g.add_nodes(
    list(range(len(showed_test_adj_matrix))), # Создает список номеров вершин
    title=[str(x) for x in range(len(showed_test_adj_matrix))], # Создает список заголовков для вершин
    label=[str(x) for x in range(len(showed_test_adj_matrix))], # Создает список меток для вершин
    color=["#171717" for _ in range(len(showed_test_adj_matrix))] # Создает список цветов для вершин
)

for a, line in enumerate(showed_test_adj_matrix): # Перебирает строки матрицы смежности, используя для получения индекса строки (a) и самой строки (line)
    for b, dist in enumerate(line): # Перебирает элементы в строке line, используя для получения индекса столбца (b) и значения элемента (dist)
        if a==b:
            continue
        if dist: # Если вес ребра больше 0 или имеет другое значение
            g.add_edge(a, b, weight=dist) # То, добавляет ребро между вершинами a и b в граф g

g.show("graph.html") # Генерирует HTML-файл с визуализацией графа и сохраняет его
os.system("graph.html") # Открывает HTML-файл в браузере по умолчанию


,Алгоритм,Ноды,Время (ns)
0,Флойда-Уоршелла,11,1300
1,Дейкстры,11,600
2,Беллмана-Форда,11,400
3,Джонсона,11,400
4,Левита,11,400
5,Йена,11,400


graph.html


0

### Выполнение:

### Вывод